<a href="https://colab.research.google.com/github/jetsonmom/halla_ai/blob/main/%EC%98%81%EC%83%81_DETECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 먼저 필요한 패키지 설치
!pip install ultralytics
!pip install pytube
!pip install opencv-python

In [ ]:

# 필요한 패키지 설치
!pip install -q ultralytics yt-dlp opencv-python

In [ ]:
# 필요한 패키지 설치
!pip install -q ultralytics yt-dlp opencv-python

from ultralytics import YOLO
import cv2
import numpy as np
import os
import json
from datetime import datetime
import sys
import time
import subprocess

# Google Colab 환경인지 확인
try:
    from google.colab import drive
    from google.colab.patches import cv2_imshow
    IN_COLAB = True
    print("Google Colab 환경에서 실행 중")
except:
    IN_COLAB = False
    print("로컬 환경에서 실행 중")

# 현재 작업 디렉토리 설정
current_dir = os.getcwd()
print(f"현재 작업 디렉토리: {current_dir}")

# 결과 저장을 위한 폴더 생성
save_dir = os.path.join(current_dir, "youtube_yolo_results")
os.makedirs(save_dir, exist_ok=True)
os.makedirs(os.path.join(save_dir, "images"), exist_ok=True)

print(f"결과 저장 디렉토리: {save_dir}")

# 타임스탬프 생성
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# YouTube 동영상 URL
youtube_url = "https://www.youtube.com/shorts/XOpJlQwXsFs"

# 모델 로드
try:
    model = YOLO('yolov8n.pt')
    print("YOLOv8 모델 로드 성공")
except Exception as e:
    print(f"모델 로드 오류: {e}")
    sys.exit(1)

# yt-dlp를 사용하여 YouTube 동영상 다운로드
try:
    print(f"YouTube 동영상 다운로드 중: {youtube_url}")
    video_file = os.path.join(save_dir, f"youtube_video_{timestamp}.mp4")

    # yt-dlp 명령 실행
    yt_dlp_cmd = [
        "yt-dlp",
        youtube_url,
        "-o", video_file,
        "-f", "best[ext=mp4]",
        "--no-playlist"
    ]

    print(f"실행 명령: {' '.join(yt_dlp_cmd)}")

    # 명령 실행 및 출력 캡처
    result = subprocess.run(yt_dlp_cmd, capture_output=True, text=True)

    # 성공 여부 확인
    if result.returncode != 0:
        print(f"다운로드 오류: {result.stderr}")
        raise Exception(f"yt-dlp 다운로드 실패: {result.stderr}")

    print(f"다운로드 완료: {video_file}")

    # 파일이 존재하는지 확인
    if not os.path.exists(video_file):
        raise Exception(f"다운로드된 파일을 찾을 수 없음: {video_file}")

    print(f"파일 크기: {os.path.getsize(video_file) / (1024*1024):.2f} MB")

except Exception as e:
    print(f"YouTube 동영상 다운로드 오류: {e}")
    sys.exit(1)

# 비디오 파일 열기
cap = cv2.VideoCapture(video_file)
if not cap.isOpened():
    print(f"다운로드한 비디오 파일을 열 수 없습니다: {video_file}")
    sys.exit(1)

# 비디오 정보 출력
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"비디오 해상도: {width}x{height}, FPS: {fps}, 총 프레임: {total_frames}")

# 결과 비디오 설정
video_output_path = os.path.join(save_dir, f"detection_{timestamp}.mp4")
print(f"결과 비디오 저장 경로: {video_output_path}")

try:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_output_path, fourcc, fps, (width, height))

    if not out.isOpened():
        raise Exception("비디오 저장 실패")

except Exception as e:
    print(f"비디오 출력 설정 오류: {e}")
    try:
        # 대체 코덱 시도
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        video_output_path = os.path.join(save_dir, f"detection_{timestamp}.avi")
        out = cv2.VideoWriter(video_output_path, fourcc, fps, (width, height))

        if not out.isOpened():
            raise Exception("대체 코덱도 실패")

    except Exception as e2:
        print(f"모든 비디오 코덱 시도 실패: {e2}")
        out = None

# 탐지 결과 저장용 리스트
detection_data = []
frame_counter = 0

print("객체 인식 시작...")
start_time = time.time()

# 진행 상태 표시 간격 설정
progress_interval = max(1, total_frames // 20)  # 전체의 5% 마다 표시

# Shorts는 짧은 영상이므로 모든 프레임을 처리해도 무방
try:
    while cap.isOpened():
        success, frame = cap.read()

        if not success:
            break

        # 진행 상태 표시
        if frame_counter % progress_interval == 0 or frame_counter == total_frames - 1:
            progress = (frame_counter / total_frames) * 100
            elapsed_time = time.time() - start_time
            estimated_total = (elapsed_time / (frame_counter + 1)) * total_frames
            remaining_time = max(0, estimated_total - elapsed_time)

            print(f"진행 중: {progress:.1f}% ({frame_counter}/{total_frames}) - 남은 시간: {remaining_time:.1f}초")

        # YOLOv8 객체 인식
        results = model(frame)

        # 결과 시각화
        annotated_frame = results[0].plot()

        # 인식 결과 저장
        frame_detections = []

        # 인식된 객체 정보 추출
        for r in results:
            boxes = r.boxes
            for box in boxes:
                # 클래스, 신뢰도, 위치 정보
                cls_id = int(box.cls[0])
                class_name = model.names[cls_id]
                confidence = float(box.conf[0])

                # 박스 좌표
                x1, y1, x2, y2 = box.xyxy[0].tolist()

                # 탐지 정보 저장
                detection_info = {
                    "frame": frame_counter,
                    "class": class_name,
                    "confidence": round(confidence, 3),
                    "bbox": [round(x1), round(y1), round(x2), round(y2)]
                }
                frame_detections.append(detection_info)

        # 프레임에 탐지된 객체가 있으면 데이터 저장
        if frame_detections:
            detection_data.append({
                "frame_number": frame_counter,
                "detections": frame_detections
            })

            # 로그 출력 (짧은 동영상이므로 모든 감지 결과 표시)
            for det in frame_detections:
                print(f"프레임 {frame_counter}: {det['class']} 발견, 확률 {det['confidence']:.2f}")

        # 모든 프레임 이미지 저장 (Shorts는 짧으므로)
        img_path = os.path.join(save_dir, "images", f"frame_{frame_counter:04d}.jpg")
        cv2.imwrite(img_path, annotated_frame)

        # 결과 비디오에 프레임 추가
        if out is not None:
            out.write(annotated_frame)

        # Colab에서는 주요 프레임 표시
        if IN_COLAB and frame_counter % max(1, total_frames // 5) == 0:
            print(f"프레임 {frame_counter} 미리보기:")
            cv2_imshow(annotated_frame)

        frame_counter += 1

except Exception as e:
    print(f"처리 중 오류 발생: {e}")

finally:
    # 처리 결과 JSON으로 저장
    json_output_path = os.path.join(save_dir, f"detections_{timestamp}.json")
    with open(json_output_path, 'w') as f:
        json.dump(detection_data, f, indent=4)

    # 리소스 해제
    cap.release()
    if out is not None:
        out.release()
    cv2.destroyAllWindows()

# 처리 시간 계산
total_time = time.time() - start_time
fps_processing = frame_counter / total_time

# 처리 통계 출력
total_detections = sum(len(frame["detections"]) for frame in detection_data)
unique_classes = set()
for frame in detection_data:
    for det in frame["detections"]:
        unique_classes.add(det["class"])

print("\n========== 처리 완료 ==========")
print(f"처리된 총 프레임: {frame_counter}")
print(f"인식된 총 객체 수: {total_detections}")
print(f"인식된 객체 종류: {', '.join(unique_classes)}")
print(f"총 처리 시간: {total_time:.2f}초 (평균 {fps_processing:.2f} FPS)")
print(f"\n결과 파일:")
print(f"- 원본 비디오: {video_file}")
print(f"- 인식 결과 비디오: {video_output_path if out is not None else '저장 실패'}")
print(f"- 인식 데이터 JSON: {json_output_path}")
print(f"- 프레임 이미지: {os.path.join(save_dir, 'images')}")
print("================================")

# 결과 요약 표시 (Colab용)
if IN_COLAB and frame_counter > 0:
    # 인식된 객체별 통계
    class_counts = {}
    for frame in detection_data:
        for det in frame["detections"]:
            class_name = det["class"]
            if class_name in class_counts:
                class_counts[class_name] += 1
            else:
                class_counts[class_name] = 1

    print("\n인식된 객체 통계:")
    for cls, count in sorted(class_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"- {cls}: {count}개")

    # 첫 번째, 중간, 마지막 프레임 보여주기
    if os.path.exists(os.path.join(save_dir, "images")):
        image_files = sorted([f for f in os.listdir(os.path.join(save_dir, "images")) if f.endswith('.jpg')])
        if image_files:
            samples = [0, len(image_files)//2, -1]  # 첫 번째, 중간, 마지막
            for i, idx in enumerate(samples):
                if 0 <= idx < len(image_files):
                    print(f"\n{['첫 번째', '중간', '마지막'][i]} 프레임:")
                    img_path = os.path.join(save_dir, "images", image_files[idx])
                    img = cv2.imread(img_path)